<a href="https://colab.research.google.com/github/vgaurav3011/Average-Screen-Time-Calculation-of-Actors-Cartoon-Characters/blob/master/Final_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls

gdrive	sample_data


In [0]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!ls
!unzip -qq 'tiny-imagenet-200.zip'
!ls

--2019-04-18 06:29:25--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  17.8MB/s    in 15s     

2019-04-18 06:29:41 (15.6 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]

gdrive	sample_data  tiny-imagenet-200.zip
gdrive	sample_data  tiny-imagenet-200	tiny-imagenet-200.zip


In [0]:
import numpy as np
import pandas as pd
from keras import initializers
from keras.models import Model
from keras import layers
import tensorflow as tf
import scipy.ndimage as scnd
import matplotlib.pyplot as plt
%matplotlib  inline
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Convolution2D, MaxPooling2D, BatchNormalization, ReLU, LeakyReLU

Using TensorFlow backend.


In [0]:
val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)

,File,Class
0,val_0.JPEG,n03444034
1,val_1.JPEG,n04067472
2,val_2.JPEG,n04070727


In [0]:
val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', 
                       header=None, 
                       names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)
#These are functions to be passed as a preprocessing function in ImageDataGenerator
from skimage import exposure, color
def AHE(img):
    img_adapteq = exposure.equalize_adapthist(img, clip_limit=0.03)
    return img_adapteq
def HE(img):
    img_eq = exposure.equalize_hist(img)
    return img_eq
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=20,
                                  zoom_range=0.15,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  fill_mode="nearest")

train_generator = train_datagen.flow_from_directory(directory = './tiny-imagenet-200/train', 
                                                    target_size=(64,64), 
                                                    batch_size=512,
                                                    class_mode='categorical')
valid_datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=20,
                                 fill_mode="nearest")
val_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64, 64),
                                                    color_mode='rgb', class_mode='categorical', batch_size=500, shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
def residual_block(y, nb_channels, kernel_size, init):

    shortcut = y

    y = layers.SeparableConv2D(nb_channels, kernel_size=kernel_size, strides=(1, 1), padding='same', kernel_initializer=init)(y)
    y = layers.BatchNormalization()(y)
    y = layers.LeakyReLU()(y)

    y = layers.Conv2D(nb_channels, kernel_size=kernel_size, strides=(1, 1), padding='same', kernel_initializer=init)(y)
    y = layers.BatchNormalization()(y)
    
    y = layers.add([shortcut, y])
    y = layers.BatchNormalization()(y)
    y = layers.LeakyReLU()(y)
    

    return y

In [0]:
def resnet_3():
    init = initializers.glorot_normal(seed=0)
    img_height = 64
    img_width = 64
    channel = 3
    input = Input(shape=(img_height, img_width, channel,))
    x = layers.Conv2D(256, kernel_size=(3, 3), strides=(2, 2), padding='same', kernel_initializer=init, input_shape=(64,64,3))(input)
    x = layers.Conv2D(256, kernel_size=(3, 3), strides=(1,1), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.SeparableConv2D(64, kernel_size=(3, 3), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 64, (3, 3), init)
    x = layers.SeparableConv2D(64, kernel_size=(3,3), padding='same', kernel_initializer=init)(x)
    x = layers.MaxPooling2D(pool_size=2, strides = (2,2))(x)
    x = layers.Conv2D(128, kernel_size=(1, 1), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 128, (3, 3), init)
    x = layers.SeparableConv2D(128, kernel_size=(3,3), padding='same', kernel_initializer=init)(x)
    x = layers.MaxPooling2D(pool_size=2, strides = (2,2))(x)
    x = layers.SeparableConv2D(128, kernel_size=(1,1), padding='same', kernel_initializer=init)(x)

    
    x = layers.Conv2D(256, kernel_size=(1, 1), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 256, (3, 3), init)
    x = layers.SeparableConv2D(256, kernel_size=(1,1), strides=2, padding='same', kernel_initializer=init)(x)
    x = layers.Conv2D(200, kernel_size=(1,1), activation = 'relu')(x)
    x = layers.MaxPooling2D(pool_size=2, strides = (2,2))(x)
    x = layers.Conv2D(64, kernel_size=(1, 1), padding='same', kernel_initializer=init)(x)
    x = layers.Conv2D(64, kernel_size=(1, 1), padding='same', kernel_initializer=init)(x)
    x = layers.Conv2D(200, kernel_size=(1,1), strides=2, activation = 'relu')(x)
    x = layers.SeparableConv2D(200, kernel_size=(1,1), strides=2, activation = 'softmax')(x)
    x = layers.Flatten()(x)
    
    model = Model(inputs=input, outputs=x)
    return model

In [0]:
!pip install h5py pyyaml 
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

checkpoint_path = "/content/gdrive/My Drive/check.ckpt"
#checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=False,
                                                 verbose=1,period=5)

In [0]:
from keras.layers import Input
from keras import optimizers
model = resnet_3()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 256)  7168        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 32, 256)  590080      conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 256)  1024        conv2d_2[0][0]                   
_____________________________________

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)
from keras.optimizers import SGD
sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 64, validation_data=val_generator, validation_steps=10000 // 64, epochs=10, max_queue_size=64*2,verbose=1, callbacks = [reduce_lr, cp_callback])

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
1562/1562 [==============================] - 775s 496ms/step - loss: 5.2449 - acc: 0.0075 - val_loss: 5.1573 - val_acc: 0.0108
Epoch 2/10
1562/1562 [==============================] - 708s 453ms/step - loss: 5.0921 - acc: 0.0146 - val_loss: 5.0335 - val_acc: 0.0203
Epoch 3/10
1562/1562 [==============================] - 714s 457ms/step - loss: 4.9184 - acc: 0.0283 - val_loss: 4.8382 - val_acc: 0.0384
Epoch 4/10
1562/1562 [==============================] - 714s 457ms/step - loss: 4.7520 - acc: 0.0447 - val_loss: 4.6767 - val_acc: 0.0500
Epoch 5/10
1562/1562 [==============================] - 714s 457ms/step - loss: 4.6354 - acc: 0.0550 - val_loss: 4.5273 - val_acc: 0.0675

Epoch 00005: saving model to /content/gdrive/My Drive/checkpoint.ckpt
Epoch 6/10
1562/1562 [==============================] - 710s 455ms/step - loss: 4.5210 - acc: 0.0686 - val_loss: 

In [0]:
from keras.models import load_model

model = load_model(checkpoint_path)


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [0]:
from keras.callbacks import *
#Cyclic Learning Rate
class CyclicLR(Callback):
  

    def __init__(self, base_lr=0.00000005, max_lr=0.0000009, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [0]:
clr_triangular = CyclicLR(mode='triangular2')

In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 64, validation_data=val_generator, validation_steps=10000 // 64, epochs=25, max_queue_size=64*2,verbose=1, callbacks = [reduce_lr, cp_callback, clr_triangular])

Epoch 1/25
1562/1562 [==============================] - 772s 495ms/step - loss: 4.0513 - acc: 0.1321 - val_loss: 3.9655 - val_acc: 0.1411
Epoch 2/25
1562/1562 [==============================] - 722s 462ms/step - loss: 4.0305 - acc: 0.1335 - val_loss: 3.8600 - val_acc: 0.1552
Epoch 3/25
1562/1562 [==============================] - 724s 463ms/step - loss: 3.9346 - acc: 0.1475 - val_loss: 3.8058 - val_acc: 0.1657
Epoch 4/25
1562/1562 [==============================] - 725s 464ms/step - loss: 3.9660 - acc: 0.1426 - val_loss: 3.8968 - val_acc: 0.1515
Epoch 5/25
1562/1562 [==============================] - 723s 463ms/step - loss: 3.9124 - acc: 0.1514 - val_loss: 3.7069 - val_acc: 0.1819

Epoch 00005: saving model to /content/gdrive/My Drive/checkpoint.ckpt
Epoch 6/25
1562/1562 [==============================] - 718s 460ms/step - loss: 3.8586 - acc: 0.1585 - val_loss: 3.8122 - val_acc: 0.1645
Epoch 7/25
1562/1562 [==============================] - 724s 463ms/step - loss: 3.9015 - acc: 0.1523 

In [0]:
from keras.models import load_model
from keras import backend as k
model = load_model(checkpoint_path)
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.005,
                              patience=5, min_lr=0.0000000005)

In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 64, validation_data=val_generator, validation_steps=10000 // 64, epochs=10, max_queue_size=64*2,verbose=1, callbacks = [reduce_lr, cp_callback])

Epoch 1/10
1562/1562 [==============================] - 752s 482ms/step - loss: 3.5713 - acc: 0.2022 - val_loss: 3.4647 - val_acc: 0.2212
Epoch 2/10
1562/1562 [==============================] - 682s 437ms/step - loss: 3.5548 - acc: 0.2060 - val_loss: 3.4587 - val_acc: 0.2226
Epoch 3/10
1562/1562 [==============================] - 682s 436ms/step - loss: 3.5437 - acc: 0.2078 - val_loss: 3.4638 - val_acc: 0.2239
Epoch 4/10
1562/1562 [==============================] - 674s 431ms/step - loss: 3.5347 - acc: 0.2105 - val_loss: 3.4423 - val_acc: 0.2247
Epoch 5/10
1562/1562 [==============================] - 679s 434ms/step - loss: 3.5147 - acc: 0.2128 - val_loss: 3.4257 - val_acc: 0.2273

Epoch 00005: saving model to /content/gdrive/My Drive/checkpoint.ckpt
Epoch 6/10
1562/1562 [==============================] - 679s 435ms/step - loss: 3.5016 - acc: 0.2153 - val_loss: 3.4209 - val_acc: 0.2297
Epoch 7/10
1562/1562 [==============================] - 679s 435ms/step - loss: 3.4873 - acc: 0.2173 

In [0]:
from keras.models import load_model
from keras import backend as k
model = load_model(checkpoint_path)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 64, validation_data=val_generator, validation_steps=10000 // 64, epochs=10, max_queue_size=64*2,verbose=1, callbacks = [clr_triangular, cp_callback])

Epoch 1/10
1562/1562 [==============================] - 741s 475ms/step - loss: 3.4707 - acc: 0.2191 - val_loss: 3.4981 - val_acc: 0.2145
Epoch 2/10
1562/1562 [==============================] - 700s 448ms/step - loss: 3.5109 - acc: 0.2154 - val_loss: 3.3610 - val_acc: 0.2416
Epoch 3/10
1562/1562 [==============================] - 693s 444ms/step - loss: 3.3967 - acc: 0.2342 - val_loss: 3.3840 - val_acc: 0.2357
Epoch 4/10
1562/1562 [==============================] - 679s 435ms/step - loss: 3.4959 - acc: 0.2149 - val_loss: 3.4209 - val_acc: 0.2239
Epoch 5/10
1562/1562 [==============================] - 673s 431ms/step - loss: 3.4175 - acc: 0.2299 - val_loss: 3.2529 - val_acc: 0.2607

Epoch 00005: saving model to /content/gdrive/My Drive/checkpoint.ckpt
Epoch 6/10
1562/1562 [==============================] - 677s 433ms/step - loss: 3.3901 - acc: 0.2345 - val_loss: 3.4490 - val_acc: 0.2280
Epoch 7/10
1562/1562 [==============================] - 700s 448ms/step - loss: 3.4592 - acc: 0.2229 

In [0]:
model = load_model(checkpoint_path)

In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 64, validation_data=val_generator, validation_steps=10000 // 64, epochs=10, max_queue_size=64*2,verbose=1, callbacks = [cp_callback])

Epoch 1/10
1562/1562 [==============================] - 749s 479ms/step - loss: 3.2683 - acc: 0.2543 - val_loss: 3.2109 - val_acc: 0.2695
Epoch 2/10
1562/1562 [==============================] - 688s 441ms/step - loss: 3.2568 - acc: 0.2578 - val_loss: 3.2124 - val_acc: 0.2667
Epoch 3/10
1562/1562 [==============================] - 696s 446ms/step - loss: 3.2432 - acc: 0.2613 - val_loss: 3.1942 - val_acc: 0.2723
Epoch 4/10
1562/1562 [==============================] - 703s 450ms/step - loss: 3.2488 - acc: 0.2598 - val_loss: 3.2038 - val_acc: 0.2696
Epoch 5/10
1562/1562 [==============================] - 697s 446ms/step - loss: 3.2341 - acc: 0.2597 - val_loss: 3.2121 - val_acc: 0.2713

Epoch 00005: saving model to /content/gdrive/My Drive/checkpoint.ckpt
Epoch 6/10
1562/1562 [==============================] - 684s 438ms/step - loss: 3.2292 - acc: 0.2639 - val_loss: 3.1998 - val_acc: 0.2732
Epoch 7/10
1562/1562 [==============================] - 695s 445ms/step - loss: 3.2262 - acc: 0.2639 

In [0]:
model = load_model(checkpoint_path)

In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 64, validation_data=val_generator, validation_steps=10000 // 64, epochs=10, max_queue_size=64*2,verbose=1, callbacks = [cp_callback])

Epoch 1/10
1562/1562 [==============================] - 747s 478ms/step - loss: 3.1973 - acc: 0.2697 - val_loss: 3.1760 - val_acc: 0.2752
Epoch 2/10
1562/1562 [==============================] - 712s 456ms/step - loss: 3.1909 - acc: 0.2698 - val_loss: 3.1534 - val_acc: 0.2811
Epoch 3/10
1562/1562 [==============================] - 720s 461ms/step - loss: 3.1897 - acc: 0.2695 - val_loss: 3.1448 - val_acc: 0.2829
Epoch 4/10
1562/1562 [==============================] - 704s 451ms/step - loss: 3.1851 - acc: 0.2715 - val_loss: 3.1587 - val_acc: 0.2790
Epoch 5/10
1562/1562 [==============================] - 671s 430ms/step - loss: 3.1760 - acc: 0.2728 - val_loss: 3.1424 - val_acc: 0.2840

Epoch 00005: saving model to /content/gdrive/My Drive/checkpoint.ckpt
Epoch 6/10
1562/1562 [==============================] - 657s 421ms/step - loss: 3.1696 - acc: 0.2732 - val_loss: 3.1450 - val_acc: 0.2811
Epoch 7/10
1562/1562 [==============================] - 672s 430ms/step - loss: 3.1643 - acc: 0.2735 

In [0]:
model = load_model(checkpoint_path)

In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 64, validation_data=val_generator, validation_steps=10000 // 64, epochs=10, max_queue_size=64*2,verbose=1, callbacks = [cp_callback])

Epoch 1/10
1562/1562 [==============================] - 783s 501ms/step - loss: 3.0574 - acc: 0.2943 - val_loss: 3.0551 - val_acc: 0.3003
Epoch 2/10
1562/1562 [==============================] - 732s 469ms/step - loss: 3.0535 - acc: 0.2948 - val_loss: 3.0438 - val_acc: 0.3026
Epoch 3/10
1562/1562 [==============================] - 719s 460ms/step - loss: 3.0521 - acc: 0.2969 - val_loss: 3.0831 - val_acc: 0.2957
Epoch 4/10
1562/1562 [==============================] - 713s 457ms/step - loss: 3.0422 - acc: 0.2986 - val_loss: 3.0495 - val_acc: 0.3035
Epoch 5/10
1562/1562 [==============================] - 717s 459ms/step - loss: 3.0422 - acc: 0.2975 - val_loss: 3.0372 - val_acc: 0.3049

Epoch 00005: saving model to /content/gdrive/My Drive/checkpoint.ckpt
Epoch 6/10
1562/1562 [==============================] - 710s 455ms/step - loss: 3.0363 - acc: 0.2970 - val_loss: 3.0502 - val_acc: 0.3000
Epoch 7/10
1562/1562 [==============================] - 723s 463ms/step - loss: 3.0259 - acc: 0.3012 

In [0]:
model = load_model(checkpoint_path)

In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 64, validation_data=val_generator, validation_steps=10000 // 64, epochs=5, max_queue_size=64*2,verbose=1, callbacks = [cp_callback])

Epoch 1/5
1562/1562 [==============================] - 782s 501ms/step - loss: 3.2730 - acc: 0.2542 - val_loss: 3.3118 - val_acc: 0.2558
Epoch 2/5
1562/1562 [==============================] - 736s 471ms/step - loss: 3.2565 - acc: 0.2542 - val_loss: 3.2478 - val_acc: 0.2604
Epoch 3/5
1562/1562 [==============================] - 728s 466ms/step - loss: 3.2478 - acc: 0.2593 - val_loss: 3.2209 - val_acc: 0.2682
Epoch 4/5
1562/1562 [==============================] - 718s 460ms/step - loss: 3.2302 - acc: 0.2636 - val_loss: 3.2480 - val_acc: 0.2664
Epoch 5/5
1562/1562 [==============================] - 717s 459ms/step - loss: 3.2195 - acc: 0.2635 - val_loss: 3.2174 - val_acc: 0.2702

Epoch 00005: saving model to /content/gdrive/My Drive/checkpoint.ckpt


In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 64, validation_data=val_generator, validation_steps=10000 // 64, epochs=10, max_queue_size=64*2,verbose=1, callbacks = [cp_callback])

Epoch 1/10
1562/1562 [==============================] - 775s 496ms/step - loss: 3.2028 - acc: 0.2673 - val_loss: 3.1958 - val_acc: 0.2734
Epoch 2/10
1562/1562 [==============================] - 725s 464ms/step - loss: 3.1888 - acc: 0.2695 - val_loss: 3.1992 - val_acc: 0.2782
Epoch 3/10
1562/1562 [==============================] - 729s 467ms/step - loss: 3.1866 - acc: 0.2693 - val_loss: 3.1923 - val_acc: 0.2753
Epoch 4/10
1562/1562 [==============================] - 723s 463ms/step - loss: 3.1707 - acc: 0.2720 - val_loss: 3.2201 - val_acc: 0.2722
Epoch 5/10
1562/1562 [==============================] - 728s 466ms/step - loss: 3.1583 - acc: 0.2749 - val_loss: 3.1559 - val_acc: 0.2803

Epoch 00005: saving model to /content/gdrive/My Drive/checkpoint.ckpt
Epoch 6/10
1562/1562 [==============================] - 698s 447ms/step - loss: 3.1501 - acc: 0.2770 - val_loss: 3.1967 - val_acc: 0.2711
Epoch 7/10
1562/1562 [==============================] - 731s 468ms/step - loss: 3.1463 - acc: 0.2778 

In [0]:
from keras.models import load_model
from keras import backend as k
model = load_model(checkpoint_path)
k.set_value(model.optimizer.lr,  0.003)
k.set_value(model.optimizer.momentum,  0.9)

In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 64, validation_data=val_generator, validation_steps=10000 // 64, epochs=15, max_queue_size=64*2,verbose=1, callbacks = [cp_callback])

Epoch 1/15
1562/1562 [==============================] - 791s 506ms/step - loss: 3.0182 - acc: 0.3025 - val_loss: 3.0385 - val_acc: 0.3055
Epoch 2/15
1562/1562 [==============================] - 738s 473ms/step - loss: 2.9870 - acc: 0.3073 - val_loss: 3.0087 - val_acc: 0.3053
Epoch 3/15
1562/1562 [==============================] - 735s 470ms/step - loss: 2.9915 - acc: 0.3068 - val_loss: 2.9870 - val_acc: 0.3137
Epoch 4/15
1562/1562 [==============================] - 734s 470ms/step - loss: 2.9816 - acc: 0.3087 - val_loss: 2.9955 - val_acc: 0.3114
Epoch 5/15
1562/1562 [==============================] - 737s 472ms/step - loss: 2.9748 - acc: 0.3106 - val_loss: 2.9996 - val_acc: 0.3101

Epoch 00005: saving model to /content/gdrive/My Drive/checkpoint.ckpt
Epoch 6/15
1562/1562 [==============================] - 689s 441ms/step - loss: 2.9782 - acc: 0.3100 - val_loss: 3.0072 - val_acc: 0.3095
Epoch 7/15
1562/1562 [==============================] - 728s 466ms/step - loss: 2.9587 - acc: 0.3152 

In [0]:
from keras.models import load_model
from keras import backend as k
model = load_model(checkpoint_path)
k.set_value(model.optimizer.lr,  0.0005)
k.set_value(model.optimizer.momentum,  0.9)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 64, validation_data=val_generator, validation_steps=10000 // 64, epochs=10, max_queue_size=64*2,verbose=1, callbacks = [cp_callback])

Epoch 1/10
1562/1562 [==============================] - 745s 477ms/step - loss: 2.8065 - acc: 0.3417 - val_loss: 2.8509 - val_acc: 0.3415
Epoch 2/10
1562/1562 [==============================] - 675s 432ms/step - loss: 2.7790 - acc: 0.3481 - val_loss: 2.8455 - val_acc: 0.3444
Epoch 3/10
1562/1562 [==============================] - 679s 435ms/step - loss: 2.7687 - acc: 0.3486 - val_loss: 2.8442 - val_acc: 0.3442
Epoch 4/10
1562/1562 [==============================] - 678s 434ms/step - loss: 2.7646 - acc: 0.3506 - val_loss: 2.8431 - val_acc: 0.3449
Epoch 5/10
1562/1562 [==============================] - 679s 435ms/step - loss: 2.7578 - acc: 0.3506 - val_loss: 2.8430 - val_acc: 0.3440

Epoch 00005: saving model to /content/gdrive/My Drive/checkpoint.ckpt
Epoch 6/10
1562/1562 [==============================] - 682s 437ms/step - loss: 2.7532 - acc: 0.3515 - val_loss: 2.8447 - val_acc: 0.3444
Epoch 7/10
1562/1562 [==============================] - 685s 439ms/step - loss: 2.7431 - acc: 0.3548 

In [0]:
from keras.models import load_model
from keras import backend as k
model = load_model(checkpoint_path)
k.set_value(model.optimizer.lr,  0.0000005)
k.set_value(model.optimizer.momentum,  0.9)

In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 64, validation_data=val_generator, validation_steps=10000 // 64, epochs=10, max_queue_size=64*2,verbose=1, callbacks = [cp_callback])

Epoch 1/10
1562/1562 [==============================] - 767s 491ms/step - loss: 2.6946 - acc: 0.3635 - val_loss: 2.8055 - val_acc: 0.3527
Epoch 2/10
1562/1562 [==============================] - 713s 456ms/step - loss: 2.7025 - acc: 0.3631 - val_loss: 2.8101 - val_acc: 0.3525
Epoch 3/10
1562/1562 [==============================] - 710s 455ms/step - loss: 2.6918 - acc: 0.3636 - val_loss: 2.8079 - val_acc: 0.3523
Epoch 4/10
1562/1562 [==============================] - 706s 452ms/step - loss: 2.6988 - acc: 0.3637 - val_loss: 2.8123 - val_acc: 0.3522
Epoch 5/10
1562/1562 [==============================] - 712s 456ms/step - loss: 2.7008 - acc: 0.3629 - val_loss: 2.8074 - val_acc: 0.3524

Epoch 00005: saving model to /content/gdrive/My Drive/checkpoint.ckpt
Epoch 6/10
1562/1562 [==============================] - 701s 449ms/step - loss: 2.6966 - acc: 0.3620 - val_loss: 2.8091 - val_acc: 0.3515
Epoch 7/10
1562/1562 [==============================] - 710s 455ms/step - loss: 2.6956 - acc: 0.3636 

In [0]:
from keras.models import load_model
from keras import backend as k
model = load_model(checkpoint_path)

In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 512, validation_data=val_generator, validation_steps=10000 // 512, epochs=10, max_queue_size=64*2,verbose=1, callbacks = [ cp_callback])

Epoch 1/10
195/195 [==============================] - 102s 523ms/step - loss: 2.6983 - acc: 0.3556 - val_loss: 2.8062 - val_acc: 0.3539
Epoch 2/10
195/195 [==============================] - 89s 459ms/step - loss: 2.7051 - acc: 0.3645 - val_loss: 2.8143 - val_acc: 0.3486
Epoch 3/10
195/195 [==============================] - 89s 457ms/step - loss: 2.6747 - acc: 0.3696 - val_loss: 2.8011 - val_acc: 0.3528
Epoch 4/10
195/195 [==============================] - 88s 454ms/step - loss: 2.6828 - acc: 0.3647 - val_loss: 2.8027 - val_acc: 0.3544

Epoch 00004: saving model to /content/gdrive/My Drive/checkpoint.ckpt
Epoch 5/10
195/195 [==============================] - 75s 386ms/step - loss: 2.7150 - acc: 0.3598 - val_loss: 2.8147 - val_acc: 0.3496
Epoch 6/10
195/195 [==============================] - 90s 462ms/step - loss: 2.6836 - acc: 0.3630 - val_loss: 2.8249 - val_acc: 0.3492
Epoch 7/10
195/195 [==============================] - 90s 460ms/step - loss: 2.6655 - acc: 0.3670 - val_loss: 2.7940 -

In [0]:
from keras.models import load_model
from keras import backend as k
model = load_model(checkpoint_path)
model.save_weights('my_model_weights.h5')

In [0]:
from imgaug import augmenters as iaa

aug = iaa.SomeOf((0, None), [
        iaa.AdditiveGaussianNoise(scale=0.2),
        iaa.Add(50, per_channel=True),
        iaa.Sharpen(alpha=(0.0, 0.5), lightness=(0.5, 1.0)),
        iaa.CropAndPad(percent=(-0.1, 0.1)),
        iaa.GaussianBlur(sigma=(0.0, 1.0)),
        iaa.Emboss(alpha=(0.0, 1.0), strength=(0.5, 1.5)),
        iaa.Dropout(p=(0, 0.2)),
        iaa.CoarseDropout(0.02, size_percent=0.5),
        iaa.ElasticTransformation(alpha=(0, 1.0), sigma=0.25)
      ], random_order=True)

def generator(data):
    while(True):
        for x, y in data:
            for i in range(len(x)):
                x[i]  = aug.augment_image(x[i])
            yield (x, y)

In [0]:
model.load_weights('my_model_weights.h5')
model.fit_generator(train_generator,steps_per_epoch=100000 // 64, validation_data=val_generator, validation_steps=10000 // 64, epochs=30, max_queue_size=64*2,verbose=1, callbacks = [reduce_lr, clr_triangular, cp_callback])

Epoch 1/30
   1/1562 [..............................] - ETA: 17:46 - loss: 2.8395 - acc: 0.3906

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (1.526031). Check your callbacks.
  % delta_t_median)


   2/1562 [..............................] - ETA: 32:42 - loss: 2.7954 - acc: 0.3750

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.774012). Check your callbacks.
  % delta_t_median)


1562/1562 [==============================] - 713s 456ms/step - loss: 2.6960 - acc: 0.3660 - val_loss: 2.8101 - val_acc: 0.3535
Epoch 2/30
1562/1562 [==============================] - 642s 411ms/step - loss: 2.6990 - acc: 0.3653 - val_loss: 2.8100 - val_acc: 0.3509

Epoch 00002: saving model to /content/gdrive/My Drive/checkpoint.ckpt
Epoch 3/30
1562/1562 [==============================] - 646s 414ms/step - loss: 2.6929 - acc: 0.3645 - val_loss: 2.8109 - val_acc: 0.3513
Epoch 4/30
1562/1562 [==============================] - 662s 424ms/step - loss: 2.6962 - acc: 0.3640 - val_loss: 2.8118 - val_acc: 0.3520
Epoch 5/30
1562/1562 [==============================] - 683s 437ms/step - loss: 2.6925 - acc: 0.3633 - val_loss: 2.8086 - val_acc: 0.3519
Epoch 6/30
1562/1562 [==============================] - 692s 443ms/step - loss: 2.6921 - acc: 0.3631 - val_loss: 2.8090 - val_acc: 0.3524
Epoch 7/30
1562/1562 [==============================] - 688s 440ms/step - loss: 2.6941 - acc: 0.3625 - val_loss:

In [0]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras
model = load_model("/content/gdrive/My Drive/check.ckpt")
cp_callback = tf.keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/check.ckpt", 
                                                 save_weights_only=False,
                                                 verbose=1,period=5)
from keras import backend as k
k.set_value(model.optimizer.lr, 0.0005)
k.set_value(model.optimizer.decay, 2e-5)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        r'./tiny-imagenet-200/train/',
        target_size=(64, 64),
        batch_size=128,
        class_mode='categorical')

val_generator = valid_datagen.flow_from_dataframe(
        val_data, directory='./tiny-imagenet-200/val/images/',
        x_col='File', y_col='Class',
        target_size=(64, 64),
        batch_size=128,
        class_mode='categorical')

Instructions for updating:
Use tf.cast instead.
Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 1024, validation_data=val_generator, validation_steps=10000 // 1024, epochs=10, max_queue_size=64*2,verbose=1, callbacks = [cp_callback])

Epoch 1/10
782/782 [==============================] - 396s 506ms/step - loss: 1.8744 - acc: 0.5221 - val_loss: 2.6552 - val_acc: 0.4054
Epoch 2/10
782/782 [==============================] - 366s 468ms/step - loss: 1.5741 - acc: 0.5875 - val_loss: 2.6869 - val_acc: 0.4099
Epoch 3/10
782/782 [==============================] - 366s 468ms/step - loss: 1.5107 - acc: 0.6025 - val_loss: 2.6473 - val_acc: 0.4165
Epoch 4/10
782/782 [==============================] - 366s 468ms/step - loss: 1.4768 - acc: 0.6095 - val_loss: 2.7317 - val_acc: 0.4080
Epoch 5/10
79/79 [==============================] - 41s 525ms/step - loss: 2.6931 - acc: 0.4095

Epoch 00005: saving model to /content/gdrive/My Drive/check.ckpt
782/782 [==============================] - 381s 487ms/step - loss: 1.4602 - acc: 0.6129 - val_loss: 2.6931 - val_acc: 0.4095
Epoch 6/10
782/782 [==============================] - 357s 457ms/step - loss: 1.4456 - acc: 0.6168 - val_loss: 2.6963 - val_acc: 0.4181
Epoch 7/10
782/782 [=============

In [0]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras
model = load_model("/content/gdrive/My Drive/check.ckpt")
cp_callback = tf.keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/check.ckpt", 
                                                 save_weights_only=False,
                                                 verbose=1,period=5)
from keras import backend as k
k.set_value(model.optimizer.lr, 0.005)
k.set_value(model.optimizer.decay, 0.000025)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.3,
        zoom_range=0.5,
        rotation_range=10,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        r'./tiny-imagenet-200/train/',
        target_size=(64, 64),
        batch_size=128,
        class_mode='categorical')

val_generator = valid_datagen.flow_from_dataframe(
        val_data, directory='./tiny-imagenet-200/val/images/',
        x_col='File', y_col='Class',
        target_size=(64, 64),
        batch_size=128,
        class_mode='categorical')

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 1024, validation_data=val_generator, validation_steps=10000 // 1024, epochs=10, max_queue_size=64*2,verbose=1, callbacks = [cp_callback])

Epoch 1/10
782/782 [==============================] - 395s 505ms/step - loss: 2.4544 - acc: 0.4108 - val_loss: 2.7167 - val_acc: 0.3910
Epoch 2/10
782/782 [==============================] - 374s 479ms/step - loss: 2.1567 - acc: 0.4678 - val_loss: 2.5936 - val_acc: 0.4122
Epoch 3/10
782/782 [==============================] - 375s 479ms/step - loss: 2.0962 - acc: 0.4801 - val_loss: 2.5781 - val_acc: 0.4149
Epoch 4/10
782/782 [==============================] - 376s 480ms/step - loss: 2.0618 - acc: 0.4867 - val_loss: 2.5765 - val_acc: 0.4101
Epoch 5/10
79/79 [==============================] - 43s 542ms/step - loss: 2.5585 - acc: 0.4214

Epoch 00005: saving model to /content/gdrive/My Drive/check.ckpt
782/782 [==============================] - 412s 527ms/step - loss: 2.0451 - acc: 0.4915 - val_loss: 2.5585 - val_acc: 0.4214
Epoch 6/10
782/782 [==============================] - 356s 455ms/step - loss: 2.0290 - acc: 0.4941 - val_loss: 2.5392 - val_acc: 0.4173
Epoch 7/10
782/782 [=============

In [0]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras
model = load_model("/content/gdrive/My Drive/check.ckpt")
cp_callback = tf.keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/check.ckpt", 
                                                 save_weights_only=False,
                                                 verbose=1,period=5)
from keras import backend as k
k.set_value(model.optimizer.lr, 0.005)
k.set_value(model.optimizer.decay, 0.000025)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.3,
        zoom_range=0.5,
        rotation_range=10,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        r'./tiny-imagenet-200/train/',
        target_size=(64, 64),
        batch_size=128,
        class_mode='categorical')

val_generator = valid_datagen.flow_from_dataframe(
        val_data, directory='./tiny-imagenet-200/val/images/',
        x_col='File', y_col='Class',
        target_size=(64, 64),
        batch_size=128,
        class_mode='categorical')

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 1024, validation_data=val_generator, validation_steps=10000 // 1024, epochs=15, max_queue_size=64*2,verbose=1, callbacks = [cp_callback])

Epoch 1/15
782/782 [==============================] - 393s 502ms/step - loss: 2.3293 - acc: 0.4349 - val_loss: 2.5229 - val_acc: 0.4229
Epoch 2/15
782/782 [==============================] - 377s 483ms/step - loss: 2.0884 - acc: 0.4842 - val_loss: 2.4983 - val_acc: 0.4200
Epoch 3/15
782/782 [==============================] - 379s 484ms/step - loss: 2.0326 - acc: 0.4962 - val_loss: 2.5293 - val_acc: 0.4245
Epoch 4/15
782/782 [==============================] - 378s 483ms/step - loss: 2.0231 - acc: 0.4978 - val_loss: 2.4807 - val_acc: 0.4247
Epoch 5/15
79/79 [==============================] - 43s 546ms/step - loss: 2.5048 - acc: 0.4212

Epoch 00005: saving model to /content/gdrive/My Drive/check.ckpt
782/782 [==============================] - 422s 540ms/step - loss: 1.9932 - acc: 0.5029 - val_loss: 2.5048 - val_acc: 0.4212
Epoch 6/15
782/782 [==============================] - 359s 459ms/step - loss: 1.9936 - acc: 0.5008 - val_loss: 2.4586 - val_acc: 0.4356
Epoch 7/15
782/782 [=============

In [0]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras
model = load_model("/content/gdrive/My Drive/check.ckpt")
cp_callback = tf.keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/check.ckpt", 
                                                 save_weights_only=False,
                                                 verbose=1,period=5)
from keras import backend as k
k.set_value(model.optimizer.lr, 0.0005)
k.set_value(model.optimizer.decay, 0.000025)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.3,
        zoom_range=0.5,
        rotation_range=10,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        r'./tiny-imagenet-200/train/',
        target_size=(64, 64),
        batch_size=128,
        class_mode='categorical')

val_generator = valid_datagen.flow_from_dataframe(
        val_data, directory='./tiny-imagenet-200/val/images/',
        x_col='File', y_col='Class',
        target_size=(64, 64),
        batch_size=128,
        class_mode='categorical')

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 1024, validation_data=val_generator, validation_steps=10000 // 1024, epochs=15, max_queue_size=64*2,verbose=1, callbacks = [cp_callback])

Epoch 1/15
782/782 [==============================] - 392s 501ms/step - loss: 2.2754 - acc: 0.4441 - val_loss: 2.5236 - val_acc: 0.4181
Epoch 2/15
782/782 [==============================] - 379s 484ms/step - loss: 2.0388 - acc: 0.4934 - val_loss: 2.4698 - val_acc: 0.4309
Epoch 3/15
782/782 [==============================] - 378s 483ms/step - loss: 1.9960 - acc: 0.5031 - val_loss: 2.5060 - val_acc: 0.4243
Epoch 4/15
782/782 [==============================] - 379s 484ms/step - loss: 1.9677 - acc: 0.5099 - val_loss: 2.5021 - val_acc: 0.4218
Epoch 5/15
79/79 [==============================] - 43s 545ms/step - loss: 2.4625 - acc: 0.4338

Epoch 00005: saving model to /content/gdrive/My Drive/check.ckpt
782/782 [==============================] - 432s 553ms/step - loss: 1.9540 - acc: 0.5136 - val_loss: 2.4625 - val_acc: 0.4338
Epoch 6/15
782/782 [==============================] - 359s 459ms/step - loss: 1.9417 - acc: 0.5146 - val_loss: 2.5103 - val_acc: 0.4291
Epoch 7/15
782/782 [=============

In [0]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras
model = load_model("/content/gdrive/My Drive/check.ckpt")
cp_callback = tf.keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/check.ckpt", 
                                                 save_weights_only=False,
                                                 verbose=1,period=3)
from keras import backend as k
k.set_value(model.optimizer.lr, 0.00005)
k.set_value(model.optimizer.decay, 0.000025)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.3,
        zoom_range=0.5,
        channel_shift_range=0.5,
        rotation_range=10,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        r'./tiny-imagenet-200/train/',
        target_size=(64, 64),
        batch_size=1024,
        class_mode='categorical')

val_generator = valid_datagen.flow_from_dataframe(
        val_data, directory='./tiny-imagenet-200/val/images/',
        x_col='File', y_col='Class',
        target_size=(64, 64),
        batch_size=1024,
        class_mode='categorical')

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 2048, validation_data=val_generator, validation_steps=10000 // 2048, epochs=6, max_queue_size=64*2,verbose=1, callbacks = [cp_callback])

Epoch 1/6
196/196 [==============================] - 374s 2s/step - loss: 1.8607 - acc: 0.5350 - val_loss: 2.4919 - val_acc: 0.4340
Epoch 2/6
20/20 [==============================] - 33s 2s/step - loss: 2.4754 - acc: 0.4359

Epoch 00002: saving model to /content/gdrive/My Drive/check.ckpt
196/196 [==============================] - 369s 2s/step - loss: 1.8104 - acc: 0.5453 - val_loss: 2.4754 - val_acc: 0.4359
Epoch 3/6
196/196 [==============================] - 234s 1s/step - loss: 1.7756 - acc: 0.5529 - val_loss: 2.4687 - val_acc: 0.4352
Epoch 4/6
196/196 [==============================] - 269s 1s/step - loss: 1.7624 - acc: 0.5565 - val_loss: 2.4984 - val_acc: 0.4332
Epoch 5/6
20/20 [==============================] - 33s 2s/step - loss: 2.4637 - acc: 0.4389

Epoch 00005: saving model to /content/gdrive/My Drive/check.ckpt
196/196 [==============================] - 269s 1s/step - loss: 1.7364 - acc: 0.5605 - val_loss: 2.4637 - val_acc: 0.4389
Epoch 6/6
196/196 [=========================

In [0]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras
model = load_model("/content/gdrive/My Drive/check.ckpt")
cp_callback = tf.keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/check.ckpt", 
                                                 save_weights_only=False,
                                                 verbose=1,period=3)
from keras import backend as k
k.set_value(model.optimizer.lr, 0.000005)
k.set_value(model.optimizer.decay, 0.000025)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.3,
        zoom_range=0.5,
        channel_shift_range=0.5,
        rotation_range=10,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        r'./tiny-imagenet-200/train/',
        target_size=(64, 64),
        batch_size=512,
        class_mode='categorical')

val_generator = valid_datagen.flow_from_dataframe(
        val_data, directory='./tiny-imagenet-200/val/images/',
        x_col='File', y_col='Class',
        target_size=(64, 64),
        batch_size=512,
        class_mode='categorical')

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 512, validation_data=val_generator, validation_steps=10000 // 512, epochs=30, max_queue_size=512*2,verbose=1, callbacks = [cp_callback])

Epoch 1/30
196/196 [==============================] - 248s 1s/step - loss: 2.0409 - acc: 0.4957 - val_loss: 2.5546 - val_acc: 0.4184
Epoch 2/30
196/196 [==============================] - 266s 1s/step - loss: 1.8525 - acc: 0.5351 - val_loss: 2.5680 - val_acc: 0.4253
Epoch 3/30
20/20 [==============================] - 32s 2s/step - loss: 2.4832 - acc: 0.4403

Epoch 00003: saving model to /content/gdrive/My Drive/check.ckpt
196/196 [==============================] - 380s 2s/step - loss: 1.7973 - acc: 0.5480 - val_loss: 2.4832 - val_acc: 0.4403
Epoch 4/30
196/196 [==============================] - 232s 1s/step - loss: 1.7679 - acc: 0.5536 - val_loss: 2.4625 - val_acc: 0.4388
Epoch 5/30
196/196 [==============================] - 275s 1s/step - loss: 1.7487 - acc: 0.5572 - val_loss: 2.4923 - val_acc: 0.4356
Epoch 6/30
20/20 [==============================] - 33s 2s/step - loss: 2.5029 - acc: 0.4367

Epoch 00006: saving model to /content/gdrive/My Drive/check.ckpt
196/196 [===================

In [0]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras
model = load_model("/content/gdrive/My Drive/check.ckpt")
cp_callback = tf.keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/check.ckpt", 
                                                 save_weights_only=False,
                                                 verbose=1,period=3)
from keras import backend as k
k.set_value(model.optimizer.lr, 0.0000005)
k.set_value(model.optimizer.decay, 0.000025)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.3,
        zoom_range=0.5,
        channel_shift_range=0.5,
        rotation_range=10,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        r'./tiny-imagenet-200/train/',
        target_size=(64, 64),
        batch_size=512,
        class_mode='categorical')

val_generator = valid_datagen.flow_from_dataframe(
        val_data, directory='./tiny-imagenet-200/val/images/',
        x_col='File', y_col='Class',
        target_size=(64, 64),
        batch_size=512,
        class_mode='categorical')

Instructions for updating:
Use tf.cast instead.
Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 512, validation_data=val_generator, validation_steps=10000 // 512, epochs=30, max_queue_size=512*2,verbose=1, callbacks = [cp_callback])

Epoch 1/30
196/196 [==============================] - 335s 2s/step - loss: 1.9940 - acc: 0.5023 - val_loss: 2.5546 - val_acc: 0.4276
Epoch 2/30
196/196 [==============================] - 258s 1s/step - loss: 1.7829 - acc: 0.5493 - val_loss: 2.5191 - val_acc: 0.4300
Epoch 3/30
20/20 [==============================] - 32s 2s/step - loss: 2.5248 - acc: 0.4317

Epoch 00003: saving model to /content/gdrive/My Drive/check.ckpt
196/196 [==============================] - 281s 1s/step - loss: 1.7279 - acc: 0.5630 - val_loss: 2.5248 - val_acc: 0.4317
Epoch 4/30
196/196 [==============================] - 254s 1s/step - loss: 1.7082 - acc: 0.5658 - val_loss: 2.5232 - val_acc: 0.4322
Epoch 5/30
196/196 [==============================] - 263s 1s/step - loss: 1.6843 - acc: 0.5705 - val_loss: 2.5317 - val_acc: 0.4345
Epoch 6/30
20/20 [==============================] - 32s 2s/step - loss: 2.5076 - acc: 0.4384

Epoch 00006: saving model to /content/gdrive/My Drive/check.ckpt
196/196 [===================

In [0]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras
model = load_model("/content/gdrive/My Drive/check.ckpt")
cp_callback = tf.keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/check.ckpt", 
                                                 save_weights_only=False,
                                                 verbose=1,period=5)
from keras import backend as k
k.set_value(model.optimizer.lr, 0.00000007)
k.set_value(model.optimizer.decay, 0.00000025)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.3,
        width_shift_range=0.1,
        zoom_range=0.5,
        channel_shift_range=0.5,
        rotation_range=10,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        r'./tiny-imagenet-200/train/',
        target_size=(64, 64),
        batch_size=512,
        class_mode='categorical')

val_generator = valid_datagen.flow_from_dataframe(
        val_data, directory='./tiny-imagenet-200/val/images/',
        x_col='File', y_col='Class',
        target_size=(64, 64),
        batch_size=512,
        class_mode='categorical')

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 512, validation_data=val_generator, validation_steps=10000 // 512, epochs=30, max_queue_size=512*2,verbose=1, callbacks = [cp_callback])

Epoch 1/30
196/196 [==============================] - 284s 1s/step - loss: 2.0719 - acc: 0.4889 - val_loss: 2.5435 - val_acc: 0.4257
Epoch 2/30
196/196 [==============================] - 267s 1s/step - loss: 1.8499 - acc: 0.5338 - val_loss: 2.5000 - val_acc: 0.4284
Epoch 3/30
196/196 [==============================] - 265s 1s/step - loss: 1.8004 - acc: 0.5450 - val_loss: 2.4612 - val_acc: 0.4415
Epoch 4/30
196/196 [==============================] - 261s 1s/step - loss: 1.7769 - acc: 0.5516 - val_loss: 2.4753 - val_acc: 0.4407
Epoch 5/30
20/20 [==============================] - 32s 2s/step - loss: 2.4473 - acc: 0.4397

Epoch 00005: saving model to /content/gdrive/My Drive/check.ckpt
196/196 [==============================] - 287s 1s/step - loss: 1.7534 - acc: 0.5542 - val_loss: 2.4473 - val_acc: 0.4397
Epoch 6/30
196/196 [==============================] - 246s 1s/step - loss: 1.7327 - acc: 0.5595 - val_loss: 2.4332 - val_acc: 0.4427
Epoch 7/30
196/196 [==============================] - 

In [0]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras
model = load_model("/content/gdrive/My Drive/check.ckpt")
cp_callback = tf.keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/check.ckpt", 
                                                 save_weights_only=False,
                                                 verbose=1,period=5)
from keras import backend as k
k.set_value(model.optimizer.lr, 0.00000007)
k.set_value(model.optimizer.decay, 0.00000025)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.3,
        height_shift_range=0.1,
        width_shift_range=0.1,
        zoom_range=0.5,
        channel_shift_range=0.5,
        rotation_range=10,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        r'./tiny-imagenet-200/train/',
        target_size=(64, 64),
        batch_size=512,
        class_mode='categorical')

val_generator = valid_datagen.flow_from_dataframe(
        val_data, directory='./tiny-imagenet-200/val/images/',
        x_col='File', y_col='Class',
        target_size=(64, 64),
        batch_size=512,
        class_mode='categorical')

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 512, validation_data=val_generator, validation_steps=10000 // 512, epochs=5, max_queue_size=512*2,verbose=1, callbacks = [cp_callback])

Epoch 1/5
196/196 [==============================] - 441s 2s/step - loss: 2.1282 - acc: 0.4737 - val_loss: 2.5325 - val_acc: 0.4218
Epoch 2/5
196/196 [==============================] - 269s 1s/step - loss: 1.9266 - acc: 0.5178 - val_loss: 2.4611 - val_acc: 0.4413
Epoch 3/5
196/196 [==============================] - 271s 1s/step - loss: 1.8770 - acc: 0.5281 - val_loss: 2.4429 - val_acc: 0.4417
Epoch 4/5
196/196 [==============================] - 271s 1s/step - loss: 1.8476 - acc: 0.5343 - val_loss: 2.4283 - val_acc: 0.4468
Epoch 5/5
20/20 [==============================] - 34s 2s/step - loss: 2.4404 - acc: 0.4363

Epoch 00005: saving model to /content/gdrive/My Drive/check.ckpt
196/196 [==============================] - 312s 2s/step - loss: 1.8335 - acc: 0.5368 - val_loss: 2.4404 - val_acc: 0.4363


In [0]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras
model = load_model("/content/gdrive/My Drive/check.ckpt")
cp_callback = tf.keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/check.ckpt", 
                                                 save_weights_only=False,
                                                 verbose=1,period=10)
from keras import backend as k
k.set_value(model.optimizer.lr, 0.000000007)
k.set_value(model.optimizer.decay, 0.00000025)
# Using Shearing, Width Shifitng, Channel Shifting, Zoom, Rotation of image and Horizontal Flip for augmentation
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.3,
        width_shift_range=0.1,
        zoom_range=0.5,
        channel_shift_range=0.5,
        rotation_range=10,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        r'./tiny-imagenet-200/train/',
        target_size=(64, 64),
        batch_size=512,
        class_mode='categorical')

val_generator = valid_datagen.flow_from_dataframe(
        val_data, directory='./tiny-imagenet-200/val/images/',
        x_col='File', y_col='Class',
        target_size=(64, 64),
        batch_size=512,
        class_mode='categorical')

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 512, validation_data=val_generator, validation_steps=10000 // 512, epochs=30, max_queue_size=512*2,verbose=1, callbacks = [cp_callback])

Epoch 1/30
196/196 [==============================] - 381s 2s/step - loss: 1.7552 - acc: 0.5571 - val_loss: 2.4746 - val_acc: 0.4404
Epoch 2/30
196/196 [==============================] - 271s 1s/step - loss: 1.7275 - acc: 0.5612 - val_loss: 2.4351 - val_acc: 0.4431
Epoch 3/30
196/196 [==============================] - 272s 1s/step - loss: 1.7105 - acc: 0.5657 - val_loss: 2.4281 - val_acc: 0.4479
Epoch 4/30
196/196 [==============================] - 272s 1s/step - loss: 1.7033 - acc: 0.5672 - val_loss: 2.4426 - val_acc: 0.4475
Epoch 5/30
196/196 [==============================] - 272s 1s/step - loss: 1.6935 - acc: 0.5682 - val_loss: 2.4360 - val_acc: 0.4481
Epoch 6/30
196/196 [==============================] - 271s 1s/step - loss: 1.6843 - acc: 0.5722 - val_loss: 2.4315 - val_acc: 0.4437
Epoch 7/30
196/196 [==============================] - 272s 1s/step - loss: 1.6802 - acc: 0.5720 - val_loss: 2.4152 - val_acc: 0.4533
Epoch 8/30
20/20 [==============================] - 33s 2s/step - los

In [0]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras
model = load_model("/content/gdrive/My Drive/check.ckpt")
cp_callback = tf.keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/check.ckpt", 
                                                 save_weights_only=False,
                                                 verbose=1,period=10)
from keras import backend as k
k.set_value(model.optimizer.lr, 0.000000007)
k.set_value(model.optimizer.decay, 0.00000025)
# Using Shearing, Width Shifitng, Channel Shifting, Zoom, Rotation of image and Horizontal Flip for augmentation
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.3,
        width_shift_range=0.1,
        zoom_range=0.5,
        channel_shift_range=0.5,
        rotation_range=10,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        r'./tiny-imagenet-200/train/',
        target_size=(64, 64),
        batch_size=512,
        class_mode='categorical')

val_generator = valid_datagen.flow_from_dataframe(
        val_data, directory='./tiny-imagenet-200/val/images/',
        x_col='File', y_col='Class',
        target_size=(64, 64),
        batch_size=512,
        class_mode='categorical')

Instructions for updating:
Use tf.cast instead.
Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 512, validation_data=val_generator, validation_steps=10000 // 512, epochs=30, max_queue_size=512*2,verbose=1, callbacks = [cp_callback])

Epoch 1/30
196/196 [==============================] - 293s 1s/step - loss: 1.9811 - acc: 0.5046 - val_loss: 2.5471 - val_acc: 0.4246
Epoch 2/30
196/196 [==============================] - 272s 1s/step - loss: 1.7772 - acc: 0.5490 - val_loss: 2.4799 - val_acc: 0.4326
Epoch 3/30
196/196 [==============================] - 272s 1s/step - loss: 1.7294 - acc: 0.5587 - val_loss: 2.4828 - val_acc: 0.4399
Epoch 4/30
196/196 [==============================] - 273s 1s/step - loss: 1.7021 - acc: 0.5667 - val_loss: 2.4638 - val_acc: 0.4422
Epoch 5/30
196/196 [==============================] - 273s 1s/step - loss: 1.6794 - acc: 0.5712 - val_loss: 2.4777 - val_acc: 0.4375
Epoch 6/30
196/196 [==============================] - 273s 1s/step - loss: 1.6710 - acc: 0.5724 - val_loss: 2.4742 - val_acc: 0.4456
Epoch 7/30
196/196 [==============================] - 273s 1s/step - loss: 1.6552 - acc: 0.5754 - val_loss: 2.4939 - val_acc: 0.4415
Epoch 8/30
196/196 [==============================] - 273s 1s/step - 

In [0]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras
model = load_model("/content/gdrive/My Drive/check.ckpt")
cp_callback = tf.keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/check.ckpt", 
                                                 save_weights_only=False,
                                                 verbose=1,period=10)
from keras import backend as k
k.set_value(model.optimizer.lr, 0.0000000007)
k.set_value(model.optimizer.decay, 0.000000025)
# Using Shearing, Width Shifitng, Channel Shifting, Zoom, Rotation of image and Horizontal Flip for augmentation
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.3,
        width_shift_range=0.1,
        zoom_range=0.5,
        channel_shift_range=0.5,
        rotation_range=10,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        r'./tiny-imagenet-200/train/',
        target_size=(64, 64),
        batch_size=512,
        class_mode='categorical')

val_generator = valid_datagen.flow_from_dataframe(
        val_data, directory='./tiny-imagenet-200/val/images/',
        x_col='File', y_col='Class',
        target_size=(64, 64),
        batch_size=512,
        class_mode='categorical')

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 512, validation_data=val_generator, validation_steps=10000 // 512, epochs=30, max_queue_size=512*2,verbose=1, callbacks = [cp_callback])

Epoch 1/30
196/196 [==============================] - 288s 1s/step - loss: 1.9623 - acc: 0.5096 - val_loss: 2.5748 - val_acc: 0.4240
Epoch 2/30
196/196 [==============================] - 274s 1s/step - loss: 1.7552 - acc: 0.5522 - val_loss: 2.5409 - val_acc: 0.4266
Epoch 3/30
196/196 [==============================] - 274s 1s/step - loss: 1.7112 - acc: 0.5636 - val_loss: 2.4766 - val_acc: 0.4332
Epoch 4/30
196/196 [==============================] - 272s 1s/step - loss: 1.6831 - acc: 0.5707 - val_loss: 2.4682 - val_acc: 0.4387
Epoch 5/30
196/196 [==============================] - 269s 1s/step - loss: 1.6688 - acc: 0.5718 - val_loss: 2.4690 - val_acc: 0.4420
Epoch 6/30
196/196 [==============================] - 269s 1s/step - loss: 1.6424 - acc: 0.5791 - val_loss: 2.4352 - val_acc: 0.4487
Epoch 7/30
196/196 [==============================] - 270s 1s/step - loss: 1.6380 - acc: 0.5809 - val_loss: 2.4432 - val_acc: 0.4532
Epoch 8/30
196/196 [==============================] - 273s 1s/step - 

In [0]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras
model = load_model("/content/gdrive/My Drive/check.ckpt")
cp_callback = tf.keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/check.ckpt", 
                                                 save_weights_only=False,
                                                 verbose=1,period=10)
from keras import backend as k
k.set_value(model.optimizer.lr, 0.0000000007)
k.set_value(model.optimizer.decay, 0.000000025)
# Using Shearing, Width Shifitng, Channel Shifting, Zoom, Rotation of image and Horizontal Flip for augmentation
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.4,
        width_shift_range=0.1,
        zoom_range=0.5,
        channel_shift_range=0.5,
        rotation_range=10,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        r'./tiny-imagenet-200/train/',
        target_size=(64, 64),
        batch_size=512,
        class_mode='categorical')

val_generator = valid_datagen.flow_from_dataframe(
        val_data, directory='./tiny-imagenet-200/val/images/',
        x_col='File', y_col='Class',
        target_size=(64, 64),
        batch_size=512,
        class_mode='categorical')

OSError: ignored

In [0]:
model.fit_generator(train_generator,steps_per_epoch=100000 // 512, validation_data=val_generator, validation_steps=10000 // 512, epochs=30, max_queue_size=512*2,verbose=1, callbacks = [cp_callback])

Epoch 1/30
196/196 [==============================] - 201s 1s/step - loss: 1.9492 - acc: 0.5116 - val_loss: 2.5191 - val_acc: 0.4327
Epoch 2/30
196/196 [==============================] - 181s 921ms/step - loss: 1.7445 - acc: 0.5541 - val_loss: 2.5270 - val_acc: 0.4310
Epoch 3/30
196/196 [==============================] - 182s 931ms/step - loss: 1.6877 - acc: 0.5682 - val_loss: 2.4749 - val_acc: 0.4444
Epoch 4/30
196/196 [==============================] - 181s 923ms/step - loss: 1.6591 - acc: 0.5742 - val_loss: 2.4816 - val_acc: 0.4379
Epoch 5/30
196/196 [==============================] - 182s 929ms/step - loss: 1.6477 - acc: 0.5768 - val_loss: 2.4974 - val_acc: 0.4421
Epoch 6/30
196/196 [==============================] - 182s 930ms/step - loss: 1.6320 - acc: 0.5787 - val_loss: 2.4495 - val_acc: 0.4497
Epoch 7/30
196/196 [==============================] - 179s 915ms/step - loss: 1.6244 - acc: 0.5818 - val_loss: 2.4388 - val_acc: 0.4527
Epoch 8/30
196/196 [==============================]